In [13]:
#Importing the dependencies
import pandas as pd
import sklearn
import numpy as np

#loading data
df = pd.read_csv("hepatitis.csv")
df.head()

,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER_BIG,LIVER_FIRM,SPLEEN_PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK_PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY,Class
0,32.059452,female,no,yes,yes,yes,yes,yes,no,no,no,no,no,1.000000,48.365665,51.718598,4.182311,74.038508,no,LIVE
1,41.229532,female,no,no,no,no,no,yes,no,no,no,no,no,1.146760,63.522251,29.216549,4.030845,25.462303,no,LIVE
2,62.133896,female,yes,no,no,no,no,yes,no,no,no,no,no,1.000000,44.055037,42.183894,4.865354,36.232554,no,LIVE
3,48.209988,female,yes,no,yes,no,no,yes,no,yes,yes,no,no,0.669207,131.126032,50.817332,3.902101,42.755563,yes,DIE
4,32.699629,female,yes,no,yes,yes,yes,no,yes,no,yes,no,no,1.000000,123.179908,20.318702,4.075779,42.079171,no,LIVE


In [14]:
#splitting parameters to X and outcome to y
X= df.drop("Class",axis = 1)
y= df['Class']

In [15]:
#reorganizing DF so it is easier to hot encode
X= X[['SEX', 'STEROID', 'ANTIVIRALS', 'FATIGUE', 'MALAISE', 'ANOREXIA',
       'LIVER_BIG', 'LIVER_FIRM', 'SPLEEN_PALPABLE', 'SPIDERS', 'ASCITES',
       'VARICES','HISTOLOGY','AGE',  'BILIRUBIN', 'ALK_PHOSPHATE', 'SGOT', 'ALBUMIN', 'PROTIME']]

X.head()

,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER_BIG,LIVER_FIRM,SPLEEN_PALPABLE,SPIDERS,ASCITES,VARICES,HISTOLOGY,AGE,BILIRUBIN,ALK_PHOSPHATE,SGOT,ALBUMIN,PROTIME
0,female,no,yes,yes,yes,yes,yes,no,no,no,no,no,no,32.059452,1.000000,48.365665,51.718598,4.182311,74.038508
1,female,no,no,no,no,no,yes,no,no,no,no,no,no,41.229532,1.146760,63.522251,29.216549,4.030845,25.462303
2,female,yes,no,no,no,no,yes,no,no,no,no,no,no,62.133896,1.000000,44.055037,42.183894,4.865354,36.232554
3,female,yes,no,yes,no,no,yes,no,yes,yes,no,no,yes,48.209988,0.669207,131.126032,50.817332,3.902101,42.755563
4,female,yes,no,yes,yes,yes,no,yes,no,yes,no,no,no,32.699629,1.000000,123.179908,20.318702,4.075779,42.079171


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [17]:
#splitting columns on categorical and numiercal in order to label encode the yes and no
X_train_cat = X_train.iloc[:,0:13]
X_train_cont = X_train.iloc[:,13:19]
X_test_cat = X_test.iloc[:,0:13]
X_test_cont = X_test.iloc[:,13:19]

#applying label encoder across multiple rows
from collections import defaultdict
d= defaultdict(LabelEncoder)
X_train_cat = X_train_cat.apply((lambda x:d[x.name].fit_transform(x)))
X_test_cat = X_test_cat.apply((lambda x:d[x.name].transform(x)))



In [18]:
#rejoining the columns
X_train = np.concatenate((X_train_cat,X_train_cont), axis =1)
X_test= np.concatenate((X_test_cat,X_test_cont), axis =1)

In [22]:
#massaging y variable
#label encoding y variable for linear regression
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.937888

In [11]:
feature_names = X.columns
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.12522553992324903, 'ASCITES'),
 (0.11821102633421603, 'ALBUMIN'),
 (0.10916161070359215, 'BILIRUBIN'),
 (0.07983115466501667, 'AGE'),
 (0.07763591638368816, 'PROTIME'),
 (0.07530527848718237, 'SPIDERS'),
 (0.06184643993745618, 'VARICES'),
 (0.06177015071890558, 'ALK_PHOSPHATE'),
 (0.06013753417878884, 'SGOT'),
 (0.04143513135794756, 'MALAISE'),
 (0.04023458912510462, 'HISTOLOGY'),
 (0.036091894144476125, 'LIVER_FIRM'),
 (0.025985237630442568, 'SPLEEN_PALPABLE'),
 (0.023628171703482494, 'FATIGUE'),
 (0.017144377258747456, 'ANOREXIA'),
 (0.016228235755694327, 'SEX'),
 (0.011082400378298773, 'STEROID'),
 (0.010203576255560357, 'LIVER_BIG'),
 (0.00884173505815064, 'ANTIVIRALS')]